In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

In [2]:
data_dir = './../datasets/dataset1'  # Update this to your dataset directory

In [3]:
# Parameters
img_width, img_height = 224, 224
batch_size = 32
epochs = 15
num_classes = 5
validation_split = 0.2  # 20% of the data will be used for validation
test_split = 0.1

# Data preparation without augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split
)

# Data augmentation for training data
# datagen = ImageDataGenerator(
#     rescale=1./255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     validation_split=validation_split
# )

# Generators for training and validation
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=validation_split / (validation_split + test_split)
)

validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Data preparation for testing
test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=test_split / (validation_split + test_split)
)

test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 28103 images belonging to 5 classes.
Found 23416 images belonging to 5 classes.
Found 11708 images belonging to 5 classes.


In [4]:

# Load the ResNet50 model, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False)

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

In [5]:
from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support


def balanced_accuracy(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)
    return tf.py_function(
        lambda y_true, y_pred: balanced_accuracy_score(
            y_true.numpy(), y_pred.numpy()),
        (y_true, y_pred),
        tf.float64)


def fscore(y_true, y_pred):
    y_true = tf.cast(tf.argmax(y_true, axis=1), tf.int32)
    y_pred = tf.cast(tf.argmax(y_pred, axis=1), tf.int32)

    def compute_fscore(y_true, y_pred):
        _, _, fscore, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0)
        return fscore

    fscore = tf.py_function(
        compute_fscore, (y_true, y_pred), tf.float64)
    return fscore

In [6]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, roc_auc_score, log_loss

# Define custom F1 score metrics
def f1_macro(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)

    def compute_f1(y_true, y_pred):
        y_true = y_true.numpy()
        y_pred = y_pred.numpy()
        _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
        return f1

    f1 = tf.py_function(compute_f1, (y_true, y_pred), tf.float64)
    return f1

def f1_weighted(y_true, y_pred):
    y_true = tf.argmax(y_true, axis=1)
    y_pred = tf.argmax(y_pred, axis=1)

    def compute_f1(y_true, y_pred):
        y_true = y_true.numpy()
        y_pred = y_pred.numpy()
        _, _, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
        return f1

    f1 = tf.py_function(compute_f1, (y_true, y_pred), tf.float64)
    return f1

# Define the evaluation function
def evaluate_model(y_true, y_pred, y_pred_proba):
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Precision, Recall, F1 Score
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    
    # Confusion Matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    # ROC-AUC (One-vs-Rest)
    roc_auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')
    
    # Log Loss
    logloss = log_loss(y_true, y_pred_proba)
    
    return {
        'accuracy': accuracy,
        'precision_macro': precision_macro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'precision_weighted': precision_weighted,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'confusion_matrix': conf_matrix,
        'roc_auc': roc_auc,
        'log_loss': logloss
    }

In [7]:
loss = tf.keras.losses.BinaryCrossentropy()
learning_rate = 3.9e-5
metrics = [
    'accuracy',
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.AUC(),
    balanced_accuracy,
    fscore,
    tf.keras.metrics.Precision(),
    tf.keras.metrics.Recall()
]

# Compile the model
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=metrics)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_steps=validation_generator.samples // batch_size,
    validation_data=validation_generator,
    epochs=epochs
)

Epoch 1/15
426/878 [=============>................] - ETA: 2:07 - loss: 0.8823 - accuracy: 0.7322 - categorical_accuracy: 0.7322 - auc: 0.8866 - balanced_accuracy: 0.2641 - fscore: 0.2239 - precision: 0.7321 - recall: 0.7265

c:\Users\rohit\anaconda3\envs\AML\lib\site-packages\sklearn\metrics\_classification.py:386: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


878/878 [==============================] - 454s 510ms/step - loss: 0.8735 - accuracy: 0.7349 - categorical_accuracy: 0.7349 - auc: 0.8888 - balanced_accuracy: 0.2643 - fscore: 0.2244 - precision: 0.7349 - recall: 0.7322 - val_loss: 0.8674 - val_accuracy: 0.7349 - val_categorical_accuracy: 0.7349 - val_auc: 0.8910 - val_balanced_accuracy: 0.2647 - val_fscore: 0.2248 - val_precision: 0.7349 - val_recall: 0.7349
Epoch 2/15
878/878 [==============================] - 444s 506ms/step - loss: 0.8704 - accuracy: 0.7347 - categorical_accuracy: 0.7347 - auc: 0.8903 - balanced_accuracy: 0.2673 - fscore: 0.2268 - precision: 0.7347 - recall: 0.7347 - val_loss: 0.8727 - val_accuracy: 0.7349 - val_categorical_accuracy: 0.7349 - val_auc: 0.8919 - val_balanced_accuracy: 0.2675 - val_fscore: 0.2271 - val_precision: 0.7349 - val_recall: 0.7349
Epoch 3/15
878/878 [==============================] - ETA: 0s - loss: 0.8717 - accuracy: 0.7347 - categorical_accuracy: 0.7347 - auc: 0.8896 - balanced_accuracy: 0

In [ ]:
# Make predictions on the test set
y_true = test_generator.classes
y_pred = model.predict(test_generator, steps=test_generator.samples // batch_size)
y_pred_classes = np.argmax(y_pred, axis=1)

# Evaluate additional metrics
metrics = evaluate_model(y_true, y_pred_classes, y_pred)
print(metrics)


In [ ]:

# # Unfreeze some layers of the base model for fine-tuning
# for layer in base_model.layers[:143]:
#     layer.trainable = False
# for layer in base_model.layers[143:]:
#     layer.trainable = True

# # Recompile the model
# model.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# # Continue training
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // batch_size,
#     validation_steps=validation_generator.samples // batch_size,
#     validation_data=validation_generator,
#     epochs=epochs
# )

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')


365/365 [==============================] - 76s 208ms/step - loss: 0.8647 - accuracy: 0.7345
Test loss: 0.8646619915962219
Test accuracy: 0.734503448009491
